In [ ]:
import organize_files as of
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime

In [ ]:
db = 'AnzaBI'
db_user = 'db_Admin'
db_password = 'password'
db_host = '219.91.145.98'
db_port = '5432'
db_schema = '"Bowling_Data"'

try:
    engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}',echo=True,future=True)
except:
    print("Unable to Create Database")
else:
    print("COnnection Successful")

In [ ]:
if os.name == 'posix':
    file_loc = r"/home/shashankraj/Documents/DATA_Dump"
    log_file = file_loc+"/"+"Logs"
    final_files = file_loc+"/"+"Final_Df"
    backup = file_loc+"/"+"Backup"
    #today = str(datetime.today().date())
    trigger_file = file_loc+"/file_trigger/"+"new_data_received.txt"
else:
    file_loc=r"D:\Offline_Docs\Anza\DATA_Dump"
    log_file = file_loc+"\\"+"Logs"
    final_files = file_loc+"\\"+"Final_Df"
    backup = file_loc+"\\"+"Backup"
    #today = str(datetime.today().date())
    trigger_file = file_loc+"\\file_trigger\\"+"new_data_received.txt"

In [ ]:
schema_ref = {
    "Client Billing Descending":"client_billing","Fee Breakdown by Dept and Fee Earner":"fee_brkdn_dept_fe","Fee Summary by Dept and Fee Earner":"fee_smry_dept_fe",
    "Fees Billed":"fees_billed","Matter Source of Business inc Matter Bills (Bill Date)":"mttr_src_ref","Total Hours by Fee Earner-With Billings All":"tot_hrs_by_fe",
    "Matters Opened by FE":"mtrs_by_fe","Payment Received Analysis":"pmt_rcv_analysis"}

In [ ]:
dict_list = of.categorize_files(file_loc)
all_files = of.concat_files(dict_list, file_loc, log_file)

print(f'\nLog Files are located at {log_file}')

In [ ]:
for file in all_files.keys():
        all_files[file].columns = [cols.lower() for cols in all_files[file].columns]
        fname = final_files+"/"+file+".csv"
        if os.path.exists(fname):
            print(f'{fname} already exists')
            all_files[file].to_csv(final_files+"/"+file+".csv",header=False,mode="a",index=False)
        else:
            print(f'Creating DF {fname}')
            all_files[file].to_csv(final_files+"/"+file+".csv",mode="a",index=False)
        
        # try: 
        #     all_files[file].to_sql(schema_ref[file],con=engine,if_exists='append',schema='Bowling_Data',index=None)
        # except:
        #     print(f"***** Unable to write to Database for file [{file}]********* ")
        # else:
        #     print(f"Putting all data in Postgresql Server for  [{file}]")

In [ ]:
for csv_file in os.listdir(final_files):
    print(f'Processing {csv_file}')
    if csv_file.endswith('.csv'):
        tmp_df = pd.read_csv(final_files+"/"+csv_file)
        tmp_df['date_added'] = pd.to_datetime(tmp_df['date_added'])
        fname = backup+"/"+csv_file.split(".")[0]+".parquet.gzip"
        print(f'Name is {fname}')
        if os.path.exists(fname):
            print(f'File already Exists, Removing')
            os.remove(fname)
        tmp_df.to_parquet(fname,compression = 'gzip')
        print("*"*50)

In [ ]:
print("Performing Cleanup")

[os.remove(file_loc+"/Processed/"+file) for file in os.listdir(file_loc+"/Processed")]
[os.remove(file_loc+"/Unprocessed/"+file) for file in os.listdir(file_loc+"/Unprocessed")]
[os.remove(file_loc+"/file_trigger/"+file) for file in os.listdir(file_loc+"/file_trigger")]
[os.remove(file_loc+"/"+file) for file in os.listdir(file_loc) if file.endswith('.csv')]

In [ ]:
# df_cb = all_files['Client Billing Descending']
# df_cb.head()
# "Cengiz Peri" and pd.Timestamp(datetime.date(2022,3,2)) in df_cb["date_added"].values

In [ ]:
os.listdir(file_loc)